In [1]:
import tempfile

import pandas as pd
import csv
import numpy as np
import tensorflow as tf


In [2]:
#Load data and reduce the number of features for the baseline models

train = pd.read_csv('dataset/train.csv', dtype={'source_system_tab': str})
test = pd.read_csv('dataset/test.csv', dtype={'source_system_tab': str })
members = pd.read_csv('dataset/members.csv', dtype={'msno': str, 'city': str, 'registered_via': str})
songs = pd.read_csv('dataset/songs.csv', dtype={'genre_ids': str, 'language': str, 'song_length': int})

songs.drop(['composer', 'lyricist'], axis=1, inplace=True)

#Infer a missing value based on other features

songs.loc[605127, 'language'] = '31.0'

#Impute missing values

train.fillna(value='unknown', axis=1, inplace=True)
test.fillna(value='unknown', axis=1, inplace=True)
members.fillna(value='unknown', axis=1, inplace=True)
songs.fillna(value='unknown', axis=1, inplace=True)


In [3]:
#members['registration_init_time'].sort_values()
#members['expiration_date']

In [3]:
#Create lists of genre ids in the genre id column

genres = songs['genre_ids'].str.split('|')

#Create a dataframe that stores genre IDs across multiple columns (one genre per column)

genres = genres.apply(pd.Series).add_prefix('genre_')
genres.to_csv('dataset/genres.csv', index=False)

genres = pd.read_csv('dataset/genres.csv', dtype=str)


In [4]:
#Merge the training and test data with song and member data

train_set = train.merge(songs, on='song_id')
train_set = train_set.merge(members, on='msno')
test_set = test.merge(songs, on='song_id', how='left')
test_set = test_set.merge(members, on='msno', how='left')

#Separate the submission ids from the test set

ids = test_set['id']
test_set.drop('id', axis=1, inplace=True)

#Impute missing values in merged training and test sets

train_set.fillna(value='unknown', axis=1, inplace=True)
test_set.fillna(value='unknown', axis=1, inplace=True)

#Impute missing song lengths with an integer value to avoid errors due to conflicting data types

test_set['song_length'] = test_set['song_length'].replace('unknown', -99)


In [5]:
#Shuffle the data and split off 20% of the training set for use as a validation set

split_ratio = 0.8

train_set = train_set.sample(frac=1, random_state=6)
val_set = train_set[int(split_ratio*train_set.shape[0]):]
train_set = train_set[:int(split_ratio*train_set.shape[0])]

#Separate the labels from the training and validation sets

y_train = train_set['target']
train_set.drop('target', axis=1, inplace=True)

y_val = val_set['target']
val_set.drop('target', axis=1, inplace=True)


In [6]:
#Designate the target feature name and the features to be used in the dataset

FEATURES = ['msno', 'gender', 'city', 'bd', 'registered_via',
            'song_id', 'artist_name', 'song_length', 'language', 'genre_ids',
            'source_system_tab', 'source_screen_name', 'source_type']

LABEL = 'target'

#Use the feature_column module to input each feature column into the model

target = tf.feature_column.categorical_column_with_identity(key='target', num_buckets=2)

registered = tf.feature_column.categorical_column_with_vocabulary_list(key='registered_via',
                                                                       vocabulary_list=['7', '4', '9', '3', '13', '16'],
                                                                       dtype=tf.string,
                                                                       default_value=-99)

gender = tf.feature_column.categorical_column_with_vocabulary_list(key='gender',
                                                                   vocabulary_list=('female', 'male', 'unknown'),
                                                                   dtype=tf.string,
                                                                   default_value=-99)

city = tf.feature_column.categorical_column_with_vocabulary_list(key='city',
                                                          vocabulary_list=members['city'].unique(),
                                                          dtype=tf.string,
                                                          default_value=-99)

language = tf.feature_column.categorical_column_with_vocabulary_list(key='language',
                                                                     vocabulary_list=songs['language'].unique(),
                                                                     dtype=tf.string,
                                                                     default_value=-99)

artist = tf.feature_column.categorical_column_with_vocabulary_list(key='artist_name',
                                                                   vocabulary_list=songs['artist_name'].unique(),
                                                                   dtype=tf.string,
                                                                   default_value=-99)

tab = tf.feature_column.categorical_column_with_vocabulary_list(key='source_system_tab',
                                                                vocabulary_list=train['source_system_tab'].unique(),
                                                                dtype=tf.string,
                                                                default_value=-99)

screen = tf.feature_column.categorical_column_with_vocabulary_list(key='source_screen_name',
                                                                   vocabulary_list=train['source_screen_name'].unique(),
                                                                   dtype=tf.string,
                                                                   default_value=-99)

source = tf.feature_column.categorical_column_with_vocabulary_list(key='source_type',
                                                                   vocabulary_list=train['source_type'].unique(),
                                                                   dtype=tf.string,
                                                                   default_value=-99)

length = tf.feature_column.numeric_column(key='song_length',
                                          default_value=-1,
                                          dtype=tf.int32)

#Bucket categorical features with many unique categories using a hash table with a size of approximately (n/0.8)*2

msno = tf.feature_column.categorical_column_with_hash_bucket(key='msno',
                                                               hash_bucket_size=90000,
                                                               dtype=tf.string)

song_id = tf.feature_column.categorical_column_with_hash_bucket(key='song_id',
                                                             hash_bucket_size=6000000,
                                                             dtype=tf.string)

genre = tf.feature_column.categorical_column_with_vocabulary_list(key='genre_ids',
                                                                  vocabulary_list=genres['genre_0'].unique(),
                                                                  dtype=tf.string,
                                                                  default_value=-99)

hashed_genre = tf.feature_column.categorical_column_with_hash_bucket(key='genre_ids',
                                                                     hash_bucket_size=3000,
                                                                     dtype=tf.string)

#Perform one hot encoding on categorical features with few unique values

indicator_registered = tf.feature_column.indicator_column(registered)
indicator_gender = tf.feature_column.indicator_column(gender)
indicator_city = tf.feature_column.indicator_column(city)
indicator_genre = tf.feature_column.indicator_column(genre)
indicator_language = tf.feature_column.indicator_column(language)
indicator_tab = tf.feature_column.indicator_column(tab)
indicator_screen = tf.feature_column.indicator_column(screen)
indicator_source = tf.feature_column.indicator_column(source)

#Embed the categorical feature with <100 unique categories into dense vectors with approximately log2(n) dimensions

embedded_genre = tf.feature_column.embedding_column(genre, dimension=10)
embedded_song = tf.feature_column.embedding_column(song_id, dimension=22)
embedded_msno = tf.feature_column.embedding_column(msno, dimension=15)
embedded_artist = tf.feature_column.embedding_column(artist, dimension=18)

#Bucket member age into age ranges, with nonsensical values going into the 0-14 or the >80 buckets

age = tf.feature_column.numeric_column(key='bd',
                                       default_value=0,
                                       dtype=tf.int32)

age_bucket = tf.feature_column.bucketized_column(age, boundaries=[0, 14, 20, 30, 40, 50, 80])

#Assign features to be used in either the wide or the deep model (or both)

wide_columns = []
cross_columns = []
deep_columns = [
                indicator_gender, indicator_city, indicator_language, indicator_tab,
                indicator_screen, indicator_source, indicator_registered,
                #embedded_msno, embedded_song, embedded_artist, embedded_genre,
                #length, age_bucket
                ]


In [7]:
def build_estimator(model_dir, model_type):
    if model_type == 'wide':
        model = tf.estimator.LinearClassifier(model_dir=model_dir,
                                              feature_columns=wide_columns + cross_columns)

    elif model_type == 'deep':
        model = tf.estimator.DNNClassifier(model_dir=model_dir,
                                           feature_columns=deep_columns,
                                           hidden_units=[1024, 512, 256],
                                           optimizer=tf.train.AdamOptimizer(learning_rate=0.001,
                                                                            name='Adam'))

    elif model_type == 'combined':
        model = tf.estimator.DNNLinearCombinedClassifier(model_dir=model_dir,
                                                         linear_feature_columns=cross_columns,
                                                         dnn_feature_columns=deep_columns,
                                                         dnn_hidden_units=[100, 50])

    return model


In [8]:
def input_fn(X, y, mode, batch_size):
    print(X.shape)
    X.fillna(value='unknown', axis=1, inplace=True)    

    if mode == 'train':
        return tf.estimator.inputs.pandas_input_fn(x=pd.DataFrame({k: X[k].values for k in FEATURES}),
                                                   y=pd.Series(y.values),
                                                   batch_size=batch_size,
                                                   num_epochs=None,
                                                   shuffle=True,
                                                   num_threads=8,
                                                   target_column='target')
    
    elif mode == 'eval':
        return tf.estimator.inputs.pandas_input_fn(x = pd.DataFrame({k: X[k].values for k in FEATURES}),
                                                   y = pd.Series(y.values),
                                                   batch_size=batch_size,
                                                   num_epochs=1,
                                                   shuffle=False,
                                                   num_threads=1,
                                                   target_column='target')
    
    elif mode == 'predict':
        return tf.estimator.inputs.pandas_input_fn(x=pd.DataFrame({k: X[k].values for k in FEATURES}),
                                                   batch_size=batch_size,
                                                   num_epochs=1,
                                                   shuffle=False,
                                                   num_threads=1)
    

In [9]:
def train_model(model_dir, model_type, train_steps, X_train, y_train, X_test, y_test, batch_size):

#Create a temporary directory to store the model if no model directory argument is given

    model_dir = tempfile.mkdtemp() if not model_dir else model_dir
    
    print('build_estimator')
    model = build_estimator(model_dir, model_type)
    
    print('train start')
    model.train(input_fn=input_fn(X_train, y_train, mode='train', batch_size=batch_size),
                max_steps=train_steps)
    
#Evaluate the trained model on a separate validation set in n/batch_size steps
    
    model.evaluate(input_fn=input_fn(X_test, y_test, mode='eval', batch_size=batch_size),
                        steps=(X_test.shape[0]//batch_size + 1))

    print('end!')
    
    return model
    

In [10]:
deep_model = train_model(model_dir='model/', model_type='deep', train_steps=600000,
                         X_train=train_set, y_train=y_train,
                         X_test=val_set, y_test=y_val,
                         batch_size=100)


build_estimator
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000233E333BF98>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
train start
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into model/model.ckpt.
INFO:tensorflow:loss = 69.1867, step = 1
INFO:tensorflow:global_step/sec: 122.864
INFO:tensorflow:loss = 65.2656, step = 101 (0.819 sec)
INFO:tensorflow:global_step/sec: 136.012
INFO:tensorflow:loss = 65.5933, step = 201 (0.734 sec)
INFO:tensorflow:global_step/sec: 141.919
INFO:tensorflow:l

INFO:tensorflow:global_step/sec: 139.227
INFO:tensorflow:loss = 63.5948, step = 7801 (0.718 sec)
INFO:tensorflow:global_step/sec: 128.266
INFO:tensorflow:loss = 59.9075, step = 7901 (0.784 sec)
INFO:tensorflow:global_step/sec: 135.955
INFO:tensorflow:loss = 70.3092, step = 8001 (0.742 sec)
INFO:tensorflow:global_step/sec: 129.265
INFO:tensorflow:loss = 66.409, step = 8101 (0.767 sec)
INFO:tensorflow:global_step/sec: 128.428
INFO:tensorflow:loss = 63.3433, step = 8201 (0.779 sec)
INFO:tensorflow:global_step/sec: 130.068
INFO:tensorflow:loss = 63.9941, step = 8301 (0.769 sec)
INFO:tensorflow:global_step/sec: 137.841
INFO:tensorflow:loss = 59.2239, step = 8401 (0.725 sec)
INFO:tensorflow:global_step/sec: 146.33
INFO:tensorflow:loss = 58.7932, step = 8501 (0.683 sec)
INFO:tensorflow:global_step/sec: 138.419
INFO:tensorflow:loss = 64.4845, step = 8601 (0.722 sec)
INFO:tensorflow:global_step/sec: 132.575
INFO:tensorflow:loss = 64.4785, step = 8701 (0.754 sec)
INFO:tensorflow:global_step/sec:

INFO:tensorflow:loss = 60.2386, step = 16201 (0.692 sec)
INFO:tensorflow:global_step/sec: 136.024
INFO:tensorflow:loss = 64.4916, step = 16301 (0.730 sec)
INFO:tensorflow:global_step/sec: 130.541
INFO:tensorflow:loss = 66.3916, step = 16401 (0.773 sec)
INFO:tensorflow:global_step/sec: 131.672
INFO:tensorflow:loss = 70.4242, step = 16501 (0.753 sec)
INFO:tensorflow:global_step/sec: 133.419
INFO:tensorflow:loss = 64.7848, step = 16601 (0.749 sec)
INFO:tensorflow:global_step/sec: 130.911
INFO:tensorflow:loss = 64.4207, step = 16701 (0.764 sec)
INFO:tensorflow:global_step/sec: 138.34
INFO:tensorflow:loss = 63.6345, step = 16801 (0.722 sec)
INFO:tensorflow:global_step/sec: 132.818
INFO:tensorflow:loss = 62.8732, step = 16901 (0.753 sec)
INFO:tensorflow:global_step/sec: 131.884
INFO:tensorflow:loss = 65.3942, step = 17001 (0.758 sec)
INFO:tensorflow:global_step/sec: 126.682
INFO:tensorflow:loss = 63.7542, step = 17101 (0.789 sec)
INFO:tensorflow:global_step/sec: 130.235
INFO:tensorflow:loss 

INFO:tensorflow:loss = 65.5102, step = 24601 (0.711 sec)
INFO:tensorflow:global_step/sec: 134.003
INFO:tensorflow:loss = 66.1762, step = 24701 (0.746 sec)
INFO:tensorflow:global_step/sec: 136.703
INFO:tensorflow:loss = 58.7628, step = 24801 (0.732 sec)
INFO:tensorflow:global_step/sec: 136.315
INFO:tensorflow:loss = 64.993, step = 24901 (0.733 sec)
INFO:tensorflow:global_step/sec: 135.907
INFO:tensorflow:loss = 65.3584, step = 25001 (0.746 sec)
INFO:tensorflow:global_step/sec: 140.536
INFO:tensorflow:loss = 69.1727, step = 25101 (0.701 sec)
INFO:tensorflow:global_step/sec: 133.59
INFO:tensorflow:loss = 63.3068, step = 25201 (0.749 sec)
INFO:tensorflow:global_step/sec: 129.394
INFO:tensorflow:loss = 64.6754, step = 25301 (0.773 sec)
INFO:tensorflow:global_step/sec: 142.547
INFO:tensorflow:loss = 62.1051, step = 25401 (0.702 sec)
INFO:tensorflow:global_step/sec: 129.548
INFO:tensorflow:loss = 68.204, step = 25501 (0.773 sec)
INFO:tensorflow:global_step/sec: 139.715
INFO:tensorflow:loss = 

INFO:tensorflow:loss = 61.7187, step = 33001 (0.732 sec)
INFO:tensorflow:global_step/sec: 136.726
INFO:tensorflow:loss = 69.6023, step = 33101 (0.731 sec)
INFO:tensorflow:global_step/sec: 131.804
INFO:tensorflow:loss = 60.3641, step = 33201 (0.759 sec)
INFO:tensorflow:global_step/sec: 132.815
INFO:tensorflow:loss = 70.7784, step = 33301 (0.755 sec)
INFO:tensorflow:global_step/sec: 143.002
INFO:tensorflow:loss = 65.7631, step = 33401 (0.698 sec)
INFO:tensorflow:global_step/sec: 137.418
INFO:tensorflow:loss = 64.8964, step = 33501 (0.727 sec)
INFO:tensorflow:global_step/sec: 140.105
INFO:tensorflow:loss = 64.8329, step = 33601 (0.714 sec)
INFO:tensorflow:global_step/sec: 136.23
INFO:tensorflow:loss = 65.6778, step = 33701 (0.734 sec)
INFO:tensorflow:global_step/sec: 146.143
INFO:tensorflow:loss = 62.2027, step = 33801 (0.684 sec)
INFO:tensorflow:global_step/sec: 135.823
INFO:tensorflow:loss = 64.7655, step = 33901 (0.736 sec)
INFO:tensorflow:global_step/sec: 131.681
INFO:tensorflow:loss 

...

INFO:tensorflow:Evaluation [13196/14755]
INFO:tensorflow:Evaluation [13197/14755]
INFO:tensorflow:Evaluation [13198/14755]
INFO:tensorflow:Evaluation [13199/14755]
INFO:tensorflow:Evaluation [13200/14755]
INFO:tensorflow:Evaluation [13201/14755]
INFO:tensorflow:Evaluation [13202/14755]
INFO:tensorflow:Evaluation [13203/14755]
INFO:tensorflow:Evaluation [13204/14755]
INFO:tensorflow:Evaluation [13205/14755]
INFO:tensorflow:Evaluation [13206/14755]
INFO:tensorflow:Evaluation [13207/14755]
INFO:tensorflow:Evaluation [13208/14755]
INFO:tensorflow:Evaluation [13209/14755]
INFO:tensorflow:Evaluation [13210/14755]
INFO:tensorflow:Evaluation [13211/14755]
INFO:tensorflow:Evaluation [13212/14755]
INFO:tensorflow:Evaluation [13213/14755]
INFO:tensorflow:Evaluation [13214/14755]
INFO:tensorflow:Evaluation [13215/14755]
INFO:tensorflow:Evaluation [13216/14755]
INFO:tensorflow:Evaluation [13217/14755]
INFO:tensorflow:Evaluation [13218/14755]
INFO:tensorflow:Evaluation [13219/14755]
INFO:tensorflow:

INFO:tensorflow:Evaluation [13594/14755]
INFO:tensorflow:Evaluation [13595/14755]
INFO:tensorflow:Evaluation [13596/14755]
INFO:tensorflow:Evaluation [13597/14755]
INFO:tensorflow:Evaluation [13598/14755]
INFO:tensorflow:Evaluation [13599/14755]
INFO:tensorflow:Evaluation [13600/14755]
INFO:tensorflow:Evaluation [13601/14755]
INFO:tensorflow:Evaluation [13602/14755]
INFO:tensorflow:Evaluation [13603/14755]
INFO:tensorflow:Evaluation [13604/14755]
INFO:tensorflow:Evaluation [13605/14755]
INFO:tensorflow:Evaluation [13606/14755]
INFO:tensorflow:Evaluation [13607/14755]
INFO:tensorflow:Evaluation [13608/14755]
INFO:tensorflow:Evaluation [13609/14755]
INFO:tensorflow:Evaluation [13610/14755]
INFO:tensorflow:Evaluation [13611/14755]
INFO:tensorflow:Evaluation [13612/14755]
INFO:tensorflow:Evaluation [13613/14755]
INFO:tensorflow:Evaluation [13614/14755]
INFO:tensorflow:Evaluation [13615/14755]
INFO:tensorflow:Evaluation [13616/14755]
INFO:tensorflow:Evaluation [13617/14755]
INFO:tensorflow:

INFO:tensorflow:Evaluation [13794/14755]
INFO:tensorflow:Evaluation [13795/14755]
INFO:tensorflow:Evaluation [13796/14755]
INFO:tensorflow:Evaluation [13797/14755]
INFO:tensorflow:Evaluation [13798/14755]
INFO:tensorflow:Evaluation [13799/14755]
INFO:tensorflow:Evaluation [13800/14755]
INFO:tensorflow:Evaluation [13801/14755]
INFO:tensorflow:Evaluation [13802/14755]
INFO:tensorflow:Evaluation [13803/14755]
INFO:tensorflow:Evaluation [13804/14755]
INFO:tensorflow:Evaluation [13805/14755]
INFO:tensorflow:Evaluation [13806/14755]
INFO:tensorflow:Evaluation [13807/14755]
INFO:tensorflow:Evaluation [13808/14755]
INFO:tensorflow:Evaluation [13809/14755]
INFO:tensorflow:Evaluation [13810/14755]
INFO:tensorflow:Evaluation [13811/14755]
INFO:tensorflow:Evaluation [13812/14755]
INFO:tensorflow:Evaluation [13813/14755]
INFO:tensorflow:Evaluation [13814/14755]
INFO:tensorflow:Evaluation [13815/14755]
INFO:tensorflow:Evaluation [13816/14755]
INFO:tensorflow:Evaluation [13817/14755]
INFO:tensorflow:

INFO:tensorflow:Evaluation [14193/14755]
INFO:tensorflow:Evaluation [14194/14755]
INFO:tensorflow:Evaluation [14195/14755]
INFO:tensorflow:Evaluation [14196/14755]
INFO:tensorflow:Evaluation [14197/14755]
INFO:tensorflow:Evaluation [14198/14755]
INFO:tensorflow:Evaluation [14199/14755]
INFO:tensorflow:Evaluation [14200/14755]
INFO:tensorflow:Evaluation [14201/14755]
INFO:tensorflow:Evaluation [14202/14755]
INFO:tensorflow:Evaluation [14203/14755]
INFO:tensorflow:Evaluation [14204/14755]
INFO:tensorflow:Evaluation [14205/14755]
INFO:tensorflow:Evaluation [14206/14755]
INFO:tensorflow:Evaluation [14207/14755]
INFO:tensorflow:Evaluation [14208/14755]
INFO:tensorflow:Evaluation [14209/14755]
INFO:tensorflow:Evaluation [14210/14755]
INFO:tensorflow:Evaluation [14211/14755]
INFO:tensorflow:Evaluation [14212/14755]
INFO:tensorflow:Evaluation [14213/14755]
INFO:tensorflow:Evaluation [14214/14755]
INFO:tensorflow:Evaluation [14215/14755]
INFO:tensorflow:Evaluation [14216/14755]
INFO:tensorflow:

INFO:tensorflow:Evaluation [14592/14755]
INFO:tensorflow:Evaluation [14593/14755]
INFO:tensorflow:Evaluation [14594/14755]
INFO:tensorflow:Evaluation [14595/14755]
INFO:tensorflow:Evaluation [14596/14755]
INFO:tensorflow:Evaluation [14597/14755]
INFO:tensorflow:Evaluation [14598/14755]
INFO:tensorflow:Evaluation [14599/14755]
INFO:tensorflow:Evaluation [14600/14755]
INFO:tensorflow:Evaluation [14601/14755]
INFO:tensorflow:Evaluation [14602/14755]
INFO:tensorflow:Evaluation [14603/14755]
INFO:tensorflow:Evaluation [14604/14755]
INFO:tensorflow:Evaluation [14605/14755]
INFO:tensorflow:Evaluation [14606/14755]
INFO:tensorflow:Evaluation [14607/14755]
INFO:tensorflow:Evaluation [14608/14755]
INFO:tensorflow:Evaluation [14609/14755]
INFO:tensorflow:Evaluation [14610/14755]
INFO:tensorflow:Evaluation [14611/14755]
INFO:tensorflow:Evaluation [14612/14755]
INFO:tensorflow:Evaluation [14613/14755]
INFO:tensorflow:Evaluation [14614/14755]
INFO:tensorflow:Evaluation [14615/14755]
INFO:tensorflow:

In [11]:
results = deep_model.evaluate(input_fn=input_fn(train_set, y_train, mode='eval', batch_size=1000),
                              steps=1476)


INFO:tensorflow:Starting evaluation at 2017-11-21-09:56:17
INFO:tensorflow:Restoring parameters from model/model.ckpt-600000
INFO:tensorflow:Evaluation [1/1476]
INFO:tensorflow:Evaluation [2/1476]
INFO:tensorflow:Evaluation [3/1476]
INFO:tensorflow:Evaluation [4/1476]
INFO:tensorflow:Evaluation [5/1476]
INFO:tensorflow:Evaluation [6/1476]
INFO:tensorflow:Evaluation [7/1476]
INFO:tensorflow:Evaluation [8/1476]
INFO:tensorflow:Evaluation [9/1476]
INFO:tensorflow:Evaluation [10/1476]
INFO:tensorflow:Evaluation [11/1476]
INFO:tensorflow:Evaluation [12/1476]
INFO:tensorflow:Evaluation [13/1476]
INFO:tensorflow:Evaluation [14/1476]
INFO:tensorflow:Evaluation [15/1476]
INFO:tensorflow:Evaluation [16/1476]
INFO:tensorflow:Evaluation [17/1476]
INFO:tensorflow:Evaluation [18/1476]
INFO:tensorflow:Evaluation [19/1476]
INFO:tensorflow:Evaluation [20/1476]
INFO:tensorflow:Evaluation [21/1476]
INFO:tensorflow:Evaluation [22/1476]
INFO:tensorflow:Evaluation [23/1476]
INFO:tensorflow:Evaluation [24/14

INFO:tensorflow:Evaluation [213/1476]
INFO:tensorflow:Evaluation [214/1476]
INFO:tensorflow:Evaluation [215/1476]
INFO:tensorflow:Evaluation [216/1476]
INFO:tensorflow:Evaluation [217/1476]
INFO:tensorflow:Evaluation [218/1476]
INFO:tensorflow:Evaluation [219/1476]
INFO:tensorflow:Evaluation [220/1476]
INFO:tensorflow:Evaluation [221/1476]
INFO:tensorflow:Evaluation [222/1476]
INFO:tensorflow:Evaluation [223/1476]
INFO:tensorflow:Evaluation [224/1476]
INFO:tensorflow:Evaluation [225/1476]
INFO:tensorflow:Evaluation [226/1476]
INFO:tensorflow:Evaluation [227/1476]
INFO:tensorflow:Evaluation [228/1476]
INFO:tensorflow:Evaluation [229/1476]
INFO:tensorflow:Evaluation [230/1476]
INFO:tensorflow:Evaluation [231/1476]
INFO:tensorflow:Evaluation [232/1476]
INFO:tensorflow:Evaluation [233/1476]
INFO:tensorflow:Evaluation [234/1476]
INFO:tensorflow:Evaluation [235/1476]
INFO:tensorflow:Evaluation [236/1476]
INFO:tensorflow:Evaluation [237/1476]
INFO:tensorflow:Evaluation [238/1476]
INFO:tensorf

INFO:tensorflow:Evaluation [429/1476]
INFO:tensorflow:Evaluation [430/1476]
INFO:tensorflow:Evaluation [431/1476]
INFO:tensorflow:Evaluation [432/1476]
INFO:tensorflow:Evaluation [433/1476]
INFO:tensorflow:Evaluation [434/1476]
INFO:tensorflow:Evaluation [435/1476]
INFO:tensorflow:Evaluation [436/1476]
INFO:tensorflow:Evaluation [437/1476]
INFO:tensorflow:Evaluation [438/1476]
INFO:tensorflow:Evaluation [439/1476]
INFO:tensorflow:Evaluation [440/1476]
INFO:tensorflow:Evaluation [441/1476]
INFO:tensorflow:Evaluation [442/1476]
INFO:tensorflow:Evaluation [443/1476]
INFO:tensorflow:Evaluation [444/1476]
INFO:tensorflow:Evaluation [445/1476]
INFO:tensorflow:Evaluation [446/1476]
INFO:tensorflow:Evaluation [447/1476]
INFO:tensorflow:Evaluation [448/1476]
INFO:tensorflow:Evaluation [449/1476]
INFO:tensorflow:Evaluation [450/1476]
INFO:tensorflow:Evaluation [451/1476]
INFO:tensorflow:Evaluation [452/1476]
INFO:tensorflow:Evaluation [453/1476]
INFO:tensorflow:Evaluation [454/1476]
INFO:tensorf

INFO:tensorflow:Evaluation [860/1476]
INFO:tensorflow:Evaluation [861/1476]
INFO:tensorflow:Evaluation [862/1476]
INFO:tensorflow:Evaluation [863/1476]
INFO:tensorflow:Evaluation [864/1476]
INFO:tensorflow:Evaluation [865/1476]
INFO:tensorflow:Evaluation [866/1476]
INFO:tensorflow:Evaluation [867/1476]
INFO:tensorflow:Evaluation [868/1476]
INFO:tensorflow:Evaluation [869/1476]
INFO:tensorflow:Evaluation [870/1476]
INFO:tensorflow:Evaluation [871/1476]
INFO:tensorflow:Evaluation [872/1476]
INFO:tensorflow:Evaluation [873/1476]
INFO:tensorflow:Evaluation [874/1476]
INFO:tensorflow:Evaluation [875/1476]
INFO:tensorflow:Evaluation [876/1476]
INFO:tensorflow:Evaluation [877/1476]
INFO:tensorflow:Evaluation [878/1476]
INFO:tensorflow:Evaluation [879/1476]
INFO:tensorflow:Evaluation [880/1476]
INFO:tensorflow:Evaluation [881/1476]
INFO:tensorflow:Evaluation [882/1476]
INFO:tensorflow:Evaluation [883/1476]
INFO:tensorflow:Evaluation [884/1476]
INFO:tensorflow:Evaluation [885/1476]
INFO:tensorf

INFO:tensorflow:Evaluation [1074/1476]
INFO:tensorflow:Evaluation [1075/1476]
INFO:tensorflow:Evaluation [1076/1476]
INFO:tensorflow:Evaluation [1077/1476]
INFO:tensorflow:Evaluation [1078/1476]
INFO:tensorflow:Evaluation [1079/1476]
INFO:tensorflow:Evaluation [1080/1476]
INFO:tensorflow:Evaluation [1081/1476]
INFO:tensorflow:Evaluation [1082/1476]
INFO:tensorflow:Evaluation [1083/1476]
INFO:tensorflow:Evaluation [1084/1476]
INFO:tensorflow:Evaluation [1085/1476]
INFO:tensorflow:Evaluation [1086/1476]
INFO:tensorflow:Evaluation [1087/1476]
INFO:tensorflow:Evaluation [1088/1476]
INFO:tensorflow:Evaluation [1089/1476]
INFO:tensorflow:Evaluation [1090/1476]
INFO:tensorflow:Evaluation [1091/1476]
INFO:tensorflow:Evaluation [1092/1476]
INFO:tensorflow:Evaluation [1093/1476]
INFO:tensorflow:Evaluation [1094/1476]
INFO:tensorflow:Evaluation [1095/1476]
INFO:tensorflow:Evaluation [1096/1476]
INFO:tensorflow:Evaluation [1097/1476]
INFO:tensorflow:Evaluation [1098/1476]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1285/1476]
INFO:tensorflow:Evaluation [1286/1476]
INFO:tensorflow:Evaluation [1287/1476]
INFO:tensorflow:Evaluation [1288/1476]
INFO:tensorflow:Evaluation [1289/1476]
INFO:tensorflow:Evaluation [1290/1476]
INFO:tensorflow:Evaluation [1291/1476]
INFO:tensorflow:Evaluation [1292/1476]
INFO:tensorflow:Evaluation [1293/1476]
INFO:tensorflow:Evaluation [1294/1476]
INFO:tensorflow:Evaluation [1295/1476]
INFO:tensorflow:Evaluation [1296/1476]
INFO:tensorflow:Evaluation [1297/1476]
INFO:tensorflow:Evaluation [1298/1476]
INFO:tensorflow:Evaluation [1299/1476]
INFO:tensorflow:Evaluation [1300/1476]
INFO:tensorflow:Evaluation [1301/1476]
INFO:tensorflow:Evaluation [1302/1476]
INFO:tensorflow:Evaluation [1303/1476]
INFO:tensorflow:Evaluation [1304/1476]
INFO:tensorflow:Evaluation [1305/1476]
INFO:tensorflow:Evaluation [1306/1476]
INFO:tensorflow:Evaluation [1307/1476]
INFO:tensorflow:Evaluation [1308/1476]
INFO:tensorflow:Evaluation [1309/1476]
INFO:tensorflow:Evaluatio

In [12]:
predictions = deep_model.predict(input_fn=input_fn(test_set, None, mode='predict',
                                                   batch_size=10000))

submission = list()

for row in predictions:
    submission.append(row['probabilities'][1])

pd.DataFrame(data={'id': ids,
                   'target': np.array(submission)}).to_csv('submissions/benchmark_deep.csv',
                                                           header=['id', 'target'],
                                                           index=False)


INFO:tensorflow:Restoring parameters from model/model.ckpt-600000
